In [1]:
// 📦 NuGet Package Reference: Microsoft Extensions AI
// This provides the core AI abstraction layer for .NET applications
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
// 🔗 Local Reference: OpenAI Agent Extensions
// This references the local build of the Microsoft Extensions AI Agents OpenAI library
// Contains OpenAI-specific implementations for the agent framework
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0\Microsoft.Agents.AI.OpenAI.dll"

In [3]:
// 🤖 Local Reference: Core Agent Framework
// This references the core Microsoft Extensions AI Agents library
// Provides the fundamental agent abstractions and base classes
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [4]:
// 🔌 NuGet Package Reference: OpenAI Integration
// Preview version of Microsoft Extensions AI OpenAI integration
// Provides OpenAI client and service implementations
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [5]:
// 🔧 NuGet Package Reference: Environment Variables
// DotNetEnv allows loading environment variables from .env files
// Essential for managing API keys and configuration securely
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
// 📚 Import Required Namespaces
using System;                          // Core system functionality
using System.ComponentModel;           // For Description attributes on tool functions
using System.ClientModel;             // Client model abstractions

using Microsoft.Extensions.AI;        // Core AI abstractions and interfaces
using Microsoft.Agents.AI; // Agent framework classes and interfaces
using OpenAI;                         // OpenAI client library

In [7]:
// 🔧 Import Environment Variable Loading
// DotNetEnv namespace for loading .env files
using DotNetEnv;

In [8]:
// 🔧 Load Environment Variables from .env File
// This loads configuration from a .env file located 3 directories up
// Required variables: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
Env.Load("../../../.env");

In [9]:
// 🎲 Tool Function: Random Destination Generator
// This static method will be available to the agent as a callable tool
// The [Description] attribute helps the AI understand when to use this function
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    // List of popular vacation destinations around the world
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan", 
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    // Generate random index and return selected destination
    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}


In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip"));

How about a day trip to Cape Town, South Africa? Here's a suggestion for your day:

### Morning:
- **Visit Table Mountain**: Start your day early by taking the cable car up Table Mountain for breathtaking views of Cape Town. You can hike around the top if you're feeling adventurous.

### Late Morning:
- **Explore the V&A Waterfront**: Head down to the vibrant V&A Waterfront, where you can visit shops, enjoy the local culture, and grab a coffee.

### Lunch:
- **Lunch at the Waterfront**: Choose a restaurant at the waterfront offering fresh seafood or traditional South African cuisine. Try the fish and chips or bobotie for an authentic taste.

### Afternoon:
- **Visit Robben Island**: Take a ferry from the V&A Waterfront to Robben Island, where Nelson Mandela was imprisoned. The tour provides a deep insight into South Africa's history.

### Late Afternoon:
- **Bloubergstrand Beach**: After returning, relax at Bloubergstrand Beach, famous for its stunning views of Table Mountain and the o

In [15]:
await foreach (var update in agent.RunStreamingAsync("Plan me a day trip"))
{
    Console.Write(update);
}

Here's a plan for a day trip to Sydney, Australia:

### Morning
- **Breakfast at The Rocks**: Start your day at one of the charming cafes in The Rocks district. Try some delicious avocado toast or a classic Aussie breakfast.
- **Sydney Opera House**: Take a stroll to the iconic Sydney Opera House. You can explore the exterior and capture some beautiful photos.

### Midday
- **Sydney Harbour Bridge**: Walk across the Sydney Harbour Bridge for stunning views of the harbor and the city skyline. If you're feeling adventurous, consider booking a bridge climb experience.
- **Lunch in Darling Harbour**: Head to Darling Harbour for lunch. There are plenty of restaurants and bars with waterfront views. Try some fresh seafood at one of the local eateries.

### Afternoon
- **Taronga Zoo**: Take a ferry from Circular Quay to Taronga Zoo. Spend a few hours here observing Australia's unique wildlife and enjoying the views of the city from the zoo.
  
### Evening
- **Return to Circular Quay**: Head b